    Imports

In [ ]:
import keras
import tensorflow as tf


import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from math import sin, cos, pi

import tensorflow
from keras.applications.resnet import ResNet50
from keras.layers import Conv2D, LeakyReLU, GlobalAveragePooling2D, Dropout, Dense
from keras.models import Sequential

Process keypoints and resize them to correspond to images being 224x224

In [ ]:

newX = 224
newY = 224

def getNewValueAfterResize(coordinate, originalSize, newLength=newX):
    return newLength / int(originalSize) * int(coordinate)


def getNewValueAfterResizeAsString(coordinate, originalSize, newLength=newX):
    return str(getNewValueAfterResize(coordinate, originalSize, newLength))

def writeAnnotationFile(original_file, new_file):
    file1 = open(original_file, 'r')
    lines = file1.readlines()
    newlines = [
        'left_acetabular_x,left_acetabular_y,left_femural_x,left_femural_y,right_acetabular_x,right_acetabular_y,right_femural_x,right_femural_y']
    for i in range(0, len(lines), 4):
        la = lines[i].split(",")
        lf = lines[i + 1].split(",")
        rf = lines[i + 2].split(",")
        ra = lines[i + 3].split(",")
        original_image_size_x = int(la[4])
        original_image_size_y = int(la[5])
        new_elem = getNewValueAfterResizeAsString(la[1], original_image_size_x) + ',' + getNewValueAfterResizeAsString(
            la[2], original_image_size_y) + ',' + getNewValueAfterResizeAsString(lf[1],
                                                                                 original_image_size_x) + ',' + getNewValueAfterResizeAsString(
            lf[2], original_image_size_y) + ',' + \
                   getNewValueAfterResizeAsString(rf[1], original_image_size_x) + ',' + getNewValueAfterResizeAsString(
            rf[2], original_image_size_y) + ',' + getNewValueAfterResizeAsString(ra[1], original_image_size_x) + ',' + getNewValueAfterResizeAsString(ra[2], original_image_size_y)
        print(new_elem)
        newlines.append(new_elem)
    filetest = open(new_file, "w")
    newText = '\n'.join(newlines)
    filetest.write(newText)

In [ ]:
def writeTest():
    writeAnnotationFile('AnnotationsAllImages512/annotations_test.csv', "newAnnotationsTest224Res.csv")

def writeTraining():
    writeAnnotationFile('AnnotationsAllImages512/annotations_training.csv', "newAnnotationsTraining224Res.csv")

def writeInternetTest():
    writeAnnotationFile('InternetImages/labels_internet_test.csv', "newAnnotationsTestInternet224Res.csv")

def writeInternetTraining():
    writeAnnotationFile('InternetImages/labels_internet_training.csv', "newAnnotationsTrainingInternet224Res.csv")

In [ ]:
writeTest()
writeTraining()
writeInternetTest()
writeInternetTraining()
training_annotations = pd.read_csv("newAnnotationsTraining224Res.csv")
test_annotations = pd.read_csv("newAnnotationsTest224Res.csv")

Load images from folder

In [ ]:
def convertImage(img):
    color = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resize = cv2.resize(color, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    return np.reshape(resize, (224,224,1))



In [ ]:
def load_images(directory):
    images = []
    for f in os.listdir(directory):
        images.append(convertImage(cv2.imread(os.path.join(directory,f))))
    images = np.array(images)/255.
    return images

In [ ]:
def load_keypoints(keypoint_data):
    keypoint_data_csv = pd.read_csv(keypoint_data)
    keypoint_features = []
    for idx, features in keypoint_data_csv.iterrows():
        keypoint_features.append(features)
    keypoint_features = np.array(keypoint_features, dtype=float)
    return keypoint_features


In [ ]:
train_images = np.concatenate((load_images('AnnotationsAllImages512/training'), load_images('InternetImages/train')))
images = np.concatenate((load_images('AnnotationsAllImages512/training'), load_images('InternetImages/train')))
train_keypoints = np.concatenate((load_keypoints('newAnnotationsTraining224Res.csv'), load_keypoints("newAnnotationsTrainingInternet224Res.csv")))
keypoints = np.concatenate((load_keypoints('newAnnotationsTraining224Res.csv'), load_keypoints("newAnnotationsTrainingInternet224Res.csv")))
test_images = np.concatenate((load_images('AnnotationsAllImages512/test'), load_images('InternetImages/test')))
test_keypoints=np.concatenate((load_keypoints('newAnnotationsTest224Res.csv'), load_keypoints('newAnnotationsTestInternet224Res.csv')))

Plotting predictions

In [ ]:
def plot_sample(image, keypoint, axis, title):

    image = image.reshape(224,224)
    axis.imshow(image, cmap='gray')
    axis.scatter(keypoint[0::2], keypoint[1::2], c=["r", "b", "g","m"], marker='x', s=20)
    plt.title(title)

In [ ]:
def plot_samples_and_save_them_to_folder(images, keypoints, folder):
    for i in range(len(images)):
        print(i)
        image=images[i]
        image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        image= np.reshape(image, (224, 224, 1))
        keypoint=keypoints[i]
        plt.imshow(image, cmap='gray')
        colors = itertools.cycle(["r", "b", "g","m"])
        keypoint_x=keypoint[0::2]
        keypoint_y=keypoint[1::2]
        for idx in range(4):
            plt.scatter(keypoint_x[idx], keypoint_y[idx], color=next(colors), marker='x', s=20)
        plt.savefig('./'+folder +"/"+ str(i) + ".png")
        plt.close()
        plt.show()

Rotation

In [ ]:
def rotate_augmentation(images, keypoints, rotation_angles):
    rotated_images = []
    rotated_keypoints = []
    for angle in rotation_angles:
        for angle in [angle, -angle]:
            M = cv2.getRotationMatrix2D((112,112), angle, 1.)
            angle_rad = -angle*pi/180.
            for image in images:
                rotated_image = cv2.warpAffine(image, M, (224,224), flags=cv2.INTER_CUBIC)
                rotated_images.append(rotated_image)
            for keypoint in keypoints:
                rotated_keypoint = keypoint - 112.
                for idx in range(0, len(rotated_keypoint), 2):
                    rotated_keypoint[idx] = rotated_keypoint[idx]*cos(angle_rad)-rotated_keypoint[idx+1]*sin(angle_rad)
                    rotated_keypoint[idx+1] = rotated_keypoint[idx]*sin(angle_rad)+rotated_keypoint[idx+1]*cos(angle_rad)
                rotated_keypoint += 112.
                rotated_keypoints.append(rotated_keypoint)

    return np.reshape(rotated_images,(-1,224,224,1)), rotated_keypoints

rotated_train_images, rotated_train_keypoints = rotate_augmentation(images, keypoints, aug_config.rotation_angles)
train_images = np.concatenate((train_images, rotated_train_images))
train_keypoints = np.concatenate((train_keypoints, rotated_train_keypoints))
fig, axis = plt.subplots()
plot_sample(rotated_train_images[19], rotated_train_keypoints[19], axis, "Rotation Augmentation")

Brightness alteration

In [ ]:
def alter_brightness(images, keypoints):
    altered_brightness_images = []
    inc_brightness_images = np.clip(images*1.2, 0.0, 1.0)
    dec_brightness_images = np.clip(images*0.6, 0.0, 1.0)
    altered_brightness_images.extend(inc_brightness_images)
    altered_brightness_images.extend(dec_brightness_images)
    return altered_brightness_images, np.concatenate((keypoints, keypoints))


altered_brightness_images, altered_brightness_keypoints = alter_brightness(images, keypoints)
train_images = np.concatenate((train_images, altered_brightness_images))
train_keypoints = np.concatenate((train_keypoints, altered_brightness_keypoints))
fig, axis = plt.subplots()
plot_sample(altered_brightness_images[19], altered_brightness_keypoints[19], axis, "Alter Brightness Augmentation")

Shift

In [ ]:
def shift_images(images, keypoints, pixel_shifts):
    shifted_images = []
    shifted_keypoints = []
    for shift in pixel_shifts:
        for (shift_x,shift_y) in [(-shift,-shift),(-shift,shift),(shift,-shift),(shift,shift)]:
            M = np.float32([[1,0,shift_x],[0,1,shift_y]])
            for image, keypoint in zip(images, keypoints):
                shifted_image = cv2.warpAffine(image, M, (224,224), flags=cv2.INTER_CUBIC)
                shifted_keypoint = np.array([(point+shift_x) if idx%2==0 else (point+shift_y) for idx, point in enumerate(keypoint)])
                if np.all(0.0<shifted_keypoint) and np.all(shifted_keypoint<224.0):
                    shifted_images.append(shifted_image.reshape(224,224,1))
                    shifted_keypoints.append(shifted_keypoint)
    shifted_keypoints = np.clip(shifted_keypoints,0.0,224.0)
    return shifted_images, shifted_keypoints

shifted_train_images, shifted_train_keypoints = shift_images(images, keypoints, aug_config.pixel_shifts)
train_images = np.concatenate((train_images, shifted_train_images))
train_keypoints = np.concatenate((train_keypoints, shifted_train_keypoints))
fig, axis = plt.subplots()
plot_sample(shifted_train_images[8], shifted_train_keypoints[8], axis, "Shift Augmentation")

Random noise

In [ ]:
def add_noise(images):
    noisy_images = []
    for image in images:
        noisy_image = cv2.add(image, 0.018*np.random.randn(224,224,1))    # Adding random normal noise to the input image & clip the resulting noisy image between [-1,1]
        noisy_images.append(noisy_image.reshape(224,224,1))
    return noisy_images

noisy_train_images = add_noise(images)
train_images = np.concatenate((train_images, noisy_train_images))
train_keypoints = np.concatenate((train_keypoints, keypoints))
fig, axis = plt.subplots()
plot_sample(noisy_train_images[8], keypoints[8], axis, "Random Noise Augmentation")

Horizontal flips

In [ ]:
def flip_images(images, keypoints):
    flipped_keypoints=[]
    flipped_images = []
    for image, keypoint in zip(images, keypoints):
        flipped_keypoint = np.array([(224 - point) if idx%2==0 else point for idx,point in enumerate(keypoint)])
        for idx in range(2):
            flipped_keypoint[idx], flipped_keypoint[idx+6] = flipped_keypoint[idx+6], flipped_keypoint[idx]
            flipped_keypoint[idx+2], flipped_keypoint[idx+4] = flipped_keypoint[idx+4], flipped_keypoint[idx+2]

        flipped_image=cv2.flip(image, 1)
        flipped_keypoints.append(flipped_keypoint)
        flipped_images.append(flipped_image.reshape(224,224,1))
    return flipped_images, flipped_keypoints

flipped_images, flipped_keypoints = flip_images(images, keypoints)
train_images = np.concatenate((train_images, flipped_images))
train_keypoints = np.concatenate((train_keypoints, flipped_keypoints))
fig, axis = plt.subplots()
plot_sample(flipped_images[8], flipped_keypoints[8], axis, "Horizontal flip augmentation")

Model creation

In [ ]:
from keras.applications.resnet import ResNet50

model = Sequential()
pretrained_model = ResNet50(input_shape=(224,224,3), include_top=False, weights='imagenet')
pretrained_model.trainable = False

model.add(Conv2D(3, (1,1), padding='same', input_shape=(224,224,1)))
model.add(LeakyReLU(alpha=0.3))
model.add(pretrained_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(8))
model.summary()

Training model with resnet layers frozen

In [ ]:

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.metrics import RootMeanSquaredError
earlyStopping = EarlyStopping(monitor='loss', patience=10, mode='min')
opt = tf.keras.optimizers.SGD(learning_rate=0.004, momentum=0.03, nesterov=True)
optAdam = tf.keras.optimizers.Adam(learning_rate=0.0005)
rlp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-15, mode='min', verbose=1)
from datetime import datetime

now = datetime.now()
current_time = str(now.strftime("%H:%M:%S"))
current_time=current_time.replace(":","-")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="current_time.h5",
    save_best_only=True,
    save_weights_only=True,
    monitor='val_loss',
    mode='min')
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[RootMeanSquaredError()])

history = model.fit(train_images, train_keypoints, epochs=20, batch_size=4, callbacks=[earlyStopping, rlp], validation_data=(test_images, test_keypoints))

Plot model history

In [ ]:
sns.set_style('darkgrid')

fig, ax = plt.subplots(2, 1, figsize=(20, 10))
df = pd.DataFrame(history.history)
df[['loss', 'val_loss']].plot(ax=ax[0])
df[['root_mean_squared_error', 'val_root_mean_squared_error']].plot(ax=ax[1])
ax[0].set_title('MSE')
ax[1].set_title('RMSE')
fig.suptitle('Model Metrics', fontsize=18);

Save test set predictions before fine-tuning

In [ ]:
predict = model.predict(test_images)
plot_samples_and_save_them_to_folder(test_images,predict,"ResNetTestResults/BeforeFinetuning")


Model Finetuning

In [ ]:
from keras.callbacks import ModelCheckpoint
modelWeightsFile='saveResnetFinetunedRound3.h5'
model_chpt = ModelCheckpoint(filepath=modelWeightsFile,
                                     monitor='val_loss',
                                     verbose=1,
                                     save_weights_only=True,
                                     save_best_only=True)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import RootMeanSquaredError
earlyStopping = EarlyStopping(monitor='loss', patience=10, mode='min')
optAdam = tf.keras.optimizers.Adam(learning_rate=0.0001)
rlp = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=1e-15, mode='min', verbose=1)
model.layers[2].trainable=True
model.compile(optimizer=optAdam, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

history = model.fit(train_images, train_keypoints, epochs=45, batch_size=4, callbacks=[earlyStopping, rlp, model_chpt], validation_data=(test_images, test_keypoints))

In [ ]:
predict = model.predict(test_images)
plot_samples_and_save_them_to_folder(test_images,predict,"ResNetTestResults/AfterFinetuning")

